In [1]:
import torch
from torch import nn

# Defining the train, val datasets paths for Cityscapes and GTA5
TRAIN_PATH_CityScapes = ['data/Cityscapes/Cityspaces/gtFine/train'
                , 'data/Cityscapes/Cityspaces/images/train'
              ]

VAL_PATH_CityScapes = [
                'data/Cityscapes/Cityspaces/gtFine/val',
                'data/Cityscapes/Cityspaces/images/val'
              ]

TRAIN_PATH_GTA5 = ['data/gta5_modified/GTA5_Modified/images',
              'data/gta5_modified/GTA5_Modified/labels',
              ]


PROJECT_NAME = 'MLDL_PART4_test_local'
PROJECT_RUN_NAME = 'some_test'

key = ''

with open('wandb.key','r') as key_file:
    key = str(key_file.readline())
    print('You wandb private key is : ',key)

WANDB_KEY = key

LEARNING_RATE_D = 1e-4
LEARNING_RATE = 1e-3
LEARNING_RATE_DECAY_ITER = 1
MOMENTUM = 0.9
NUM_CLASSES = 19
DIS_POWER = 0.9
GEN_POWER = 0.1
WEIGHT_DECAY = 0.0005
IGNORE_LABEL = 19
LAMBDA = 0.01
ITERATIONS = 10
EPOCHS = 50

SOURCE_BATCH_SIZE = 12
TARGET_BATCH_SIZE = 12

note = '''

    Testing the project implementation on local device

'''

config = {
    'description' : '',
    'Generator_Optimizer' : 'Adam',
    'Discriminator_Optimizer' : 'Adam',
    'Generator_context' : 'resnet18',
    'generator_lr_decay_power':GEN_POWER,
    'discriminator_lr_decay_power':DIS_POWER,
    'num_classes': NUM_CLASSES,
    'init_gen_lr': LEARNING_RATE,
    'init_dis_lr': LEARNING_RATE_D,
    'momentum': MOMENTUM,
    'weight_decay': WEIGHT_DECAY,
    'ignore_label': IGNORE_LABEL,
    'lambda': LAMBDA,
    'train_path': TRAIN_PATH_GTA5,
    'iterations' : ITERATIONS,
    'epochs' : EPOCHS,
    'gen_lr_decay_iter' : LEARNING_RATE_DECAY_ITER,
    'source_batch_size': SOURCE_BATCH_SIZE,
    'target_batch_size': TARGET_BATCH_SIZE,
}

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

You wandb private key is :  80a085c493537c92cfb05248a3dedabdecf02a15


'cuda'

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import cityscapes, gta5
import utils


# Create a transforms pipeline for Cityscapes
input_transform_Cityscapes = transforms.Compose([
    transforms.Resize((512, 1024), antialias=True), # 'antialias = True' ensures that the resizing operation uses antialiasing, which can produce better quality images when downscaling
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

output_transform_Cityscapes = transforms.Compose([
    transforms.Resize((512, 1024), antialias=True),
    utils.IntRangeTransformer(min_val=0, max_val=19)
])

# Create a transforms pipeline for GTA5
input_transform_GTA5 = transforms.Compose([
    transforms.Resize((720, 1280)),
    # put the augmentation please.
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
    ], p=0.5),  # Apply Gaussian Blur

    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

output_transform_GTA5 = transforms.Compose([
    transforms.Resize((720, 1280)),
])


# For Cityscapes
train_datasets = cityscapes.CityScapes(TRAIN_PATH_CityScapes[0], TRAIN_PATH_CityScapes[1], input_transform_Cityscapes, output_transform_Cityscapes)
val_datasets = cityscapes.CityScapes(VAL_PATH_CityScapes[0], VAL_PATH_CityScapes[1], input_transform_Cityscapes, output_transform_Cityscapes)


train_dataloader = DataLoader(train_datasets, batch_size=TARGET_BATCH_SIZE,shuffle=True, pin_memory=True, num_workers=4)
# For validation and testing, shuffling is typically set to False. 
#This ensures that the validation/testing results are reproducible and the same samples are evaluated in the same order each time.
val_dataloader = DataLoader(val_datasets, batch_size=TARGET_BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=4)

# For GTA5
train_datasetGTA5 = gta5.GTA5(TRAIN_PATH_GTA5[0], TRAIN_PATH_GTA5[1], input_transform_GTA5, output_transform_GTA5)
train_dataloaderGTA5 = DataLoader(train_datasetGTA5, batch_size=SOURCE_BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)


In [3]:
from models.domain_shift.adversarial.model import DomainDiscriminator
from models.bisenet.build_bisenet import BiSeNet

In [4]:
def forModel(model):
    if device == 'cuda':
        model = model.cuda()  # Move model to GPU if available

    # If you have multiple GPUs available
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model) # Use DataParallel to wrap your model
        
    return model

In [5]:
from callbacks import Callback
from torch import nn


import wandb
wandb.login(key=WANDB_KEY, verify=True)

class WandBCallback(Callback):
    def __init__(self, project_name : str, run_name : str =None, config : dict = None,note : str =''):
        wandb.init(project=project_name, name=run_name, config=config, notes=note)
    
    def on_train_end(self, logs=None):
        print('The train finished completely and terminate the wandb logger.')
        wandb.finish()

    def on_batch_end(self, batch, logs=None):
        wandb.log({**logs})
            
    def on_epoch_end(self, epoch,logs=None):
        wandb.log({**logs})

    def on_validation_end(self, logs=None,data=None):
        wandb.log(logs)
        wandb.log({"per class mIoU": wandb.Table(data=data)})

# defining the callbacks
callbacks = [
    WandBCallback(project_name=PROJECT_NAME,
                  run_name=PROJECT_RUN_NAME,config=config)
]

# # defining a CrossEntropyLoss for the segmentation and a BCEWithLogitsLoss for the domain classification
generator_loss = nn.CrossEntropyLoss(ignore_index=IGNORE_LABEL)
discriminator_loss = nn.BCEWithLogitsLoss()

# defining the models
generator = forModel(BiSeNet(num_classes=NUM_CLASSES,context_path=config['Generator_context'],with_interpolation=True))
# generator.load_state_dict(torch.load('/kaggle/input/weights/weightsAug1.pth'))
# defining the Discriminator
discriminator = forModel(DomainDiscriminator(num_classes=NUM_CLASSES,with_grl=False))

# defining the optimizer

if config['Generator_Optimizer'] == 'Adam':
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=LEARNING_RATE)
elif config['Generator_Optimizer'] == 'SGD':
    generator_optimizer = torch.optim.SGD(generator.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
if config['Discriminator_Optimizer'] == 'Adam':
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=LEARNING_RATE_D, betas=(0.9, 0.99), weight_decay=WEIGHT_DECAY)
elif config['Discriminator_Optimizer'] == 'SGD':
    discriminator_optimizer = torch.optim.SGD(discriminator.parameters(), lr=LEARNING_RATE_D, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sina-behnam-ity (polito_mldl). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/cs/.netrc


In [6]:
from train import val_GTA5, adversarial_train
from torch.functional import F

In [7]:
adversarial_train(
    iterations=ITERATIONS,
    epochs=EPOCHS,
    lambda_=LAMBDA,
    generator=generator,
    discriminator=discriminator,
    generator_optimizer=generator_optimizer,
    discriminator_optimizer=discriminator_optimizer,
    generator_loss=generator_loss,
    discriminator_loss=discriminator_loss,
    source_dataloader=train_dataloaderGTA5,
    target_dataloader=train_dataloader,
    gen_init_lr=LEARNING_RATE,
    dis_init_lr= LEARNING_RATE_D,
    lr_decay_iter= LEARNING_RATE_DECAY_ITER,
    max_iter=(EPOCHS * ITERATIONS),
    gen_power=GEN_POWER,
    dis_power=DIS_POWER,
    num_classes=NUM_CLASSES,
    class_names=cityscapes.class_names,
    val_loader=val_dataloader,
    do_validation=1,
    when_print=-1,
    callbacks=callbacks,
    device=device)

Epoch 0:   0%|          | 0/10 [00:00<?, ?it/s]

-------------------------------------------------- Validation --------------------------------------------------
Validation mIoU for Epoch 1: 0.0162
            Class     IoU
0            road  0.0000
1        sidewalk  0.0247
2        building  0.1882
3            wall  0.0000
4           fence  0.0005
5            pole  0.0011
6   traffic light  0.0007
7    traffic sign  0.0041
8      vegetation  0.0082
9         terrain  0.0052
10            sky  0.0379
11         person  0.0371
12          rider  0.0000
13            car  0.0003
14          truck  0.0000
15            bus  0.0000
16          train  0.0002
17     motorcycle  0.0000
18        bicycle  0.0000
----------------------------------------------------------------------------------------------------


Epoch 1:   0%|          | 0/10 [00:00<?, ?it/s]

-------------------------------------------------- Validation --------------------------------------------------
Validation mIoU for Epoch 2: 0.0153
            Class     IoU
0            road  0.0058
1        sidewalk  0.0538
2        building  0.0125
3            wall  0.0001
4           fence  0.0082
5            pole  0.0101
6   traffic light  0.0003
7    traffic sign  0.0070
8      vegetation  0.0257
9         terrain  0.0098
10            sky  0.0073
11         person  0.0142
12          rider  0.0004
13            car  0.1356
14          truck  0.0000
15            bus  0.0000
16          train  0.0002
17     motorcycle  0.0000
18        bicycle  0.0000
----------------------------------------------------------------------------------------------------


Epoch 2:   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 48:   0%|          | 0/10 [00:00<?, ?it/s]

-------------------------------------------------- Validation --------------------------------------------------
Validation mIoU for Epoch 49: 0.1716
            Class     IoU
0            road  0.6541
1        sidewalk  0.1772
2        building  0.6359
3            wall  0.0496
4           fence  0.0013
5            pole  0.0051
6   traffic light  0.0001
7    traffic sign  0.0124
8      vegetation  0.6874
9         terrain  0.1480
10            sky  0.5795
11         person  0.0002
12          rider  0.0056
13            car  0.3049
14          truck  0.0000
15            bus  0.0000
16          train  0.0000
17     motorcycle  0.0000
18        bicycle  0.0001
----------------------------------------------------------------------------------------------------


Epoch 49:   0%|          | 0/10 [00:00<?, ?it/s]

-------------------------------------------------- Validation --------------------------------------------------
Validation mIoU for Epoch 50: 0.1705
            Class     IoU
0            road  0.5762
1        sidewalk  0.2086
2        building  0.6406
3            wall  0.0502
4           fence  0.0016
5            pole  0.0046
6   traffic light  0.0001
7    traffic sign  0.0070
8      vegetation  0.6975
9         terrain  0.1682
10            sky  0.6021
11         person  0.0010
12          rider  0.0064
13            car  0.2747
14          truck  0.0000
15            bus  0.0000
16          train  0.0000
17     motorcycle  0.0000
18        bicycle  0.0000
----------------------------------------------------------------------------------------------------
The train finished completely and terminate the wandb logger.


Genrator Accuracy,▁▃▆▆▇▇▇▇▇▇▇▇█▇█████▇██████████▇█████████
dis_lr,████▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁
gen_lr,█████████▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▃▃▁
loss_adversarial,▂▂▂▂▁▁▁▁▂▁▂▂▄▃▃▃▄▅▅▄▃▃▃▅▆▅▃▄▅▆▆▆▆▅▄▆▆▇▆█
loss_disc_source,▅▅▅▅▄▄▄▄▃▃▄▅▅▆▃▂▆██▇▂▁▃▅▆▄▃▃▁▃▃▅▅▃▁▂▁▆▃▄
loss_disc_target,▆▇▇▇▇▇▇█▇█▇▇▄▆▆▇▅▄▄▆▇▇▆▄▂▄▇▇▄▃▃▄▃▄▆▃▄▃▄▁
loss_gen_source,█▅▅▄▃▄▃▃▂▃▃▃▂▃▂▂▃▃▂▂▂▂▁▂▂▂▁▁▂▁▁▁▂▂▁▁▁▂▁▁
validation_mIoU,▁▁▃▅▆▆▇▇▇▇▇▆▇▇▇▇▇▆▇██▇██▇▇▇██▇▇▇▇██▇████
Genrator Accuracy,84.13023
dis_lr,0.0
gen_lr,0.00068


In [ ]:
version = '3.0.0'
torch.save(generator.state_dict(), f'/kaggle/working/generator_v{version}_BB.pth')
torch.save(discriminator.state_dict(), f'/kaggle/working/discriminator_v{version}_BB.pth')